In [2]:
!pip install -U git+https://github.com/huggingface/transformers@de9255de27abfcae4a1f816b904915f0b1e23cd9

  Cloning https://github.com/huggingface/transformers (to revision de9255de27abfcae4a1f816b904915f0b1e23cd9) to /tmp/pip-req-build-wdo0536f
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-wdo0536f
  Running command git rev-parse -q --verify 'sha^de9255de27abfcae4a1f816b904915f0b1e23cd9'
  Running command git fetch -q https://github.com/huggingface/transformers de9255de27abfcae4a1f816b904915f0b1e23cd9
  Running command git checkout -q de9255de27abfcae4a1f816b904915f0b1e23cd9
  Resolved https://github.com/huggingface/transformers to commit de9255de27abfcae4a1f816b904915f0b1e23cd9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [49]:
import numpy as np
import pandas as pd

from transformers import AutoTokenizer, AutoModel
from transformers import pipeline

import json
import nltk
import re
from nltk.util import ngrams
nltk.download('punkt')
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [50]:
corpus = [
    "Дан двухатомный идеальный газ, молекулы которого обладают электрическим дипольным моментом d {|d| = d0 = const}. Газ находится в постоянном однородном электрическом поле e0 и имеет температуру Т. Требуется: а) найти добавок к свободной энергии рассматриваемого идеального газа, обусловленный взаимодействием дипольных моментов d с электрическим полем е0. Число молекул в газе N, объем газа V; б) зная результат пункта а), найти вклад в теплоемкость газа, обусловленный взаимодействием диполей d с полем e0; в) найти распределение по углам электрических дипольных моментов d в поле e0; г) зная результат пункта в), найти выражение для диэлектрической проницаемости газа в случае слабого поля e0.",
    "Пользуясь интегральным исчислением, решить задачи 7.12, 7.13 и 7.14 аналитически. 7.12. Пользуясь численными методами, найти положение центра масс полукруга. Для простоты расчета положить R==1,00. 7.13. Пользуясь численными методами, найти положение центра масс полушара. 7.14.Пользуясь численными методами, найти положение центра масс прямого кругового конуса высотой h=1, образующая которого составляет угол а с высотой.",
    "Заряд q распределен с постоянной поверхностной плотностью по поверхности полусферы радиуса а, лежащей в области z > 0 декартовой системы координат. Граничная окружность полусферы лежит в плоскости z = 0, центр этой окружности совпадает с началом координат. 1) Найти потенциал ф во всех точках плоскости z = 0, выразив его как функцию расстояния r до начала координат. 2) Как ориентирован вектор электрического поля в плоскости z = 0 при r < а? 3) Найти потенциал ф(z) на оси z. 4) Найти приближенно потенциал ф(r) на больших расстояниях от полусферы (|r| >> а) в присутствии дополнительного точечного заряда противоположного знака (-q), расположенного в начале координат.",
    "Материальная точка массой 0,01 кг совершает гармонические колебания с периодом 2 с. Полная энергия колеблющейся точки 10-4 Дж. Найти амплитуду колебаний, написать уравнение колебаний, найти наибольшее значение силы, действующей на точку."
          ]

# Autocomplete

#BERT MASKING NEXT TOKEN

In [5]:
# Define tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [6]:

classifier = pipeline("fill-mask", model = "DeepPavlov/rubert-base-cased-sentence", tokenizer=tokenizer)
classifier("двухатомный идеальный [MASK]")

Some weights of BertForMaskedLM were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-sentence and are newly initialized: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'score': 0.0002246604854008183,
  'token': 85588,
  'token_str': 'пени',
  'sequence': 'двухатомный идеальный пени'},
 {'score': 0.00018175203877035528,
  'token': 38132,
  'token_str': 'советы',
  'sequence': 'двухатомный идеальный советы'},
 {'score': 0.00016879251052159816,
  'token': 3813,
  'token_str': 'дан',
  'sequence': 'двухатомный идеальный дан'},
 {'score': 0.00015720840019639581,
  'token': 3111,
  'token_str': '勤',
  'sequence': 'двухатомный идеальный 勤'},
 {'score': 0.00015274326142389327,
  'token': 18349,
  'token_str': 'призван',
  'sequence': 'двухатомный идеальный призван'}]

### FAIL. But fine-tunung may help

# Text preparation for NLP

In [52]:
#prepare n-grams for QAC.

# Delete punctuation
def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

# Create dict/list of unigrams, bigrams and trigrams. dict(Json) format is for fast autocomplete library and list is for TRIE
def create_ngrams(texts, n):
  nrams_dict = dict()
  ngrams_list = []
  for text in texts:
    text = remove_punctuation(text.lower())
    tokens = word_tokenize(text)
    for i in range(n[0], n[1] + 1):
      ngram_list = list(ngrams(tokens, i))
      for ngram in ngram_list:
        nrams_dict[' '.join(ngram)] = {}
        ngrams_list.append(' '.join(ngram))
  return nrams_dict,  ngrams_list

In [53]:
n = (1, 3)
queries_dict,  query_list = create_ngrams(corpus, n)
query_list

['дан',
 'двухатомный',
 'идеальный',
 'газ',
 'молекулы',
 'которого',
 'обладают',
 'электрическим',
 'дипольным',
 'моментом',
 'd',
 'd',
 'd0',
 'const',
 'газ',
 'находится',
 'в',
 'постоянном',
 'однородном',
 'электрическом',
 'поле',
 'e0',
 'и',
 'имеет',
 'температуру',
 'т',
 'требуется',
 'а',
 'найти',
 'добавок',
 'к',
 'свободной',
 'энергии',
 'рассматриваемого',
 'идеального',
 'газа',
 'обусловленный',
 'взаимодействием',
 'дипольных',
 'моментов',
 'd',
 'с',
 'электрическим',
 'полем',
 'е0',
 'число',
 'молекул',
 'в',
 'газе',
 'n',
 'объем',
 'газа',
 'v',
 'б',
 'зная',
 'результат',
 'пункта',
 'а',
 'найти',
 'вклад',
 'в',
 'теплоемкость',
 'газа',
 'обусловленный',
 'взаимодействием',
 'диполей',
 'd',
 'с',
 'полем',
 'e0',
 'в',
 'найти',
 'распределение',
 'по',
 'углам',
 'электрических',
 'дипольных',
 'моментов',
 'd',
 'в',
 'поле',
 'e0',
 'г',
 'зная',
 'результат',
 'пункта',
 'в',
 'найти',
 'выражение',
 'для',
 'диэлектрической',
 'проницаемос

#**TRIE**

In [54]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

class Autocomplete:
    def __init__(self, vocabulary):
        self.root = TrieNode()
        self.build_trie(vocabulary)

    def build_trie(self, vocabulary):
        for word in vocabulary:
            self.insert_word(word)

    def insert_word(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end_of_word = True

    def find_words(self, prefix):
        node = self.root
        result = []
        for char in prefix:
            if char not in node.children:
                return result
            node = node.children[char]
        self.traverse_subtrie(node, prefix, result)
        return result

    def traverse_subtrie(self, node, prefix, result):
        if node.is_end_of_word:
            result.append(prefix)
        for char in node.children:
            self.traverse_subtrie(node.children[char], prefix + char, result)

In [55]:
vocabulary = query_list
autocomplete = Autocomplete(vocabulary)

In [56]:
%time
prefix = 'точк'
results = autocomplete.find_words(prefix)
print(results)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.68 µs
['точка', 'точках', 'точках плоскости', 'точках плоскости z', 'точка массой', 'точка массой 001', 'точки', 'точки 104', 'точки 104 дж', 'точку']


In [46]:
%time
prefix = 'кол'
results = autocomplete.find_words(prefix)
print(results)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.39 µs
['колебания', 'колебания с', 'колебания с периодом', 'колебаний', 'колебаний написать', 'колебаний написать уравнение', 'колебаний найти', 'колебаний найти наибольшее', 'колеблющейся', 'колеблющейся точки', 'колеблющейся точки 104']


In [57]:
%time
prefix = 'те'
results = autocomplete.find_words(prefix)
print(results)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs
['температуру', 'температуру т', 'температуру т требуется', 'теплоемкость', 'теплоемкость газа', 'теплоемкость газа обусловленный']


In [59]:
%time
prefix = 'колебаний написать'
results = autocomplete.find_words(prefix)
print(results)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs
['колебаний написать', 'колебаний написать уравнение']


### NOT BAD. Also we can try tf-idf weghted n-grams for a larger corpus

# **fast_autocomplete**

In [18]:
!pip install fast-autocomplete[levenshtein]

In [36]:
from fast_autocomplete import AutoComplete

In [40]:
words = queries_dict
autocomplete = AutoComplete(words=words, valid_chars_for_string = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя') #add characters of rus alphabet
print(autocomplete.search(word='колеба', max_cost=5, size = 5))

[['поле'], ['d в поле'], ['0 выразив его'], ['n объем газа'], ['v б зная']]


### FAIL.  not good with russian language